#### Import de-identification functions from datafly_v4.py

In [6]:
from de_id_functions import *
import numpy as np
import pylab as P
import pandas as pd
from decimal import *

#### Additional functions not included in the de_id_functions.py file.

In [7]:
def utilValues(cursor, tableName, varName):
    """
    cursor: sqlite cursor object
    tableName: string, name of sqlite table
    varName: string, name of variable to analyze
    takes values of an integer or float variable and returns the 
    mean, standard deviation, and entropy
    """
    entQry = selUnique(cursor, tableName, varName)
    entropy = shannonEntropy(entQry)
    cursor.execute("SELECT "+varName+" FROM "+tableName)
    qry = cursor.fetchall()
    qry = colToList(qry)
    qry2 = textToFloat(qry)
    if len(qry2)==0:
        print "No values could be converted to numbers"
        return
    qryArray = np.array(qry2)
    mean = qryArray.mean()
    sd = qryArray.std()
    return entropy, mean, sd

In [8]:
def binAvg(cursor, tableName, nomVarName, numVarName):
    """
    cursor: sqlite cursor object
    tableName: string, name of sqlite table
    nomVarName: string, name of variable with nominal categories
    numVarName: string, name of corresponding variable with numeric values
    For two columns, one a categorical string representation (generalization)
    of the numeric values in another column (for example column A 
    contains "10-15" and then column B contains the actual values 
    that are in that bin), will give a bin-level average of the true values
    in that bin. Designed as a tool to help improve the quality of a 
    binned (aka 'generalized') dataset. 
    """
    newVarName = nomVarName+"_avg"
    getcontext().prec = 2
    bins = selUnique(cursor,tableName,nomVarName)
    avgDic = {}
    for cat in bins:
        cursor.execute("SELECT "+numVarName+" FROM "+tableName+" WHERE "+nomVarName+" = '"+cat[0]+"'")
        qry = cursor.fetchall()
        qry = colToList(qry)
        qry2 = textToFloat(qry)
        if len(qry2)==0:
            print "No values could be converted to numbers: "+str(cat[0])
            continue
        qryArray = np.array(qry2)
        mean = qryArray.mean()
        mean = Decimal(mean)
        mean = round(mean,2)
        avgDic[cat[0]] = str(mean)
    try:
        addColumn(cursor,tableName,newVarName)
        varIndex(cursor,tableName,newVarName)
    except:
        print "column "+newVarName+" already exists, overwriting..."
        cursor.execute("UPDATE "+tableName+" SET "+newVarName+" = 'null'")
    dataUpdate(cursor,tableName,nomVarName,avgDic,True,newVarName)
        

In [9]:
def utilMatrix(cursor, tableName, varList):
    """
    cursor: sqlite cursor object
    tableName: string, name of sqlite table
    varList: list of utility variables, in format indigenous
    to this program, which is the format that results
    from the sqlite "Pragma table_info()" command.
    This function creates a Pandas dataframe/matrix of the entropy,
    mean, and standard deviation of the utility variables, 
    index is the variable name, and columns are the statistics
    """
    varNames = []
    for var in varList:
        varNames.append(var[1])
    entropies = np.array([])
    sds = np.array([])
    means = np.array([])
    uMatrix = pd.DataFrame(columns = ["Entropy","Mean","SD"], index = varNames)
    for var in varNames:
        ent, mean, sd = utilValues(cursor, tableName, var)
        uMatrix.ix[var] = [ent, mean, sd]
    return uMatrix
    

In [10]:
def textToFloat(txtList):
    """
    txtList: list of text values
    returns a list of float values, 
    skips values that cannot be converted
    """
    numList = []
    for i in txtList:
        try: numList.append(float(i))
        except: pass
    return numList

In [11]:
def lDiversity(cursor, tableName, kkeyVar, senVar):
    """
    cursor: sqlite3 cursor object                                                                                                                                                                
    tableName: string, name of main table
    kkeyVar: string, name of variable that contains concatenation of all quasi-identifiers
    senVar: string, name of variable whose value you do not want disclosed
    Checks a dataset for "l-diversity", namely that in a k-anonymous block of records
    if the sensitive value is homogeonous, then you have effectively disclosed the 
    value of the sensitive record. Bluntly sets sensitive variable to blank if not l-diverse
    """
    qry = selUnique(cursor, tableName, kkeyVar)
    for i in qry:
        cursor.execute('SELECT '+senVar+' FROM '+tableName+' WHERE '+kkeyVar+' = "'+i[0]+'" GROUP BY '+senVar)
        qry2 = cursor.fetchall()
        if len(qry2) == 1:
            cursor.execute('UPDATE '+tableName+' SET '+senVar+' = " " WHERE '+kkeyVar+' = "'+i[0]+'"')

In [12]:
def optimumDrop2(cursor, tableName, userVar, k, nonUniqueList, nComb=1):
    """                                                                                                                                                                                          
    cursor: sqlite3 cursor object                                                                                                                                                                
    tableName: string, name of main table                                                                                                                                                        
    userVar: string, name of userid var                                                                                                                                                          
    k: int, minimum cell size                                                                                                                                                                    
    nonUniqueList: list of course_combo values already cleared for k-anonymity                                                                                                                   
    nComb: int, number of courses to try to drop, default 1                                                                                                                                      
    iteratively tries 'dropping' one course for all of the records                                                                                                                               
    that are flagged as having a unique combo of courses                                                                                                                                         
    then measures the entropy of the resulting group, and                                                                                                                                        
    returns the position in courseList of the course to drop, along with the                                                                                                                     
    course_combo values that will benefit from the drop                                                                                                                                          
    """
    qry = courseUserQry(cursor, tableName, userVar, 'True')
    if len(qry)==0:
        return qry
    posLen = len(qry[0][0]) #assumes first variable in each tuple is the course combo, finds num of positions to change                                                                          
    preList = qry[:]
    preCombos = []
    for i in preList:
        preCombos.append(i[0])
    preEntropy = shannonEntropy(preList)
    postEntList = []
    preCount = 0
    for n in qry:
        preCount += n[1]
    print preCount
    iterTemp = itertools.combinations(range(posLen),nComb)
    dropCombos = []
    while True:
        try: dropCombos.append(iterTemp.next())
        except: break
    for i in dropCombos:
        #print "dropCombo:"
        #print i
        postList = []
        tmpList = qry[:]
        for j in tmpList:
            newString = ""
            for l in range(posLen):
                if l in i:
                    newString+="0"
                else:
                    newString+=j[0][l]
            postList.append((newString,j[1]))
        try:
            cursor.execute("DROP TABLE coursedrop")
            cursor.execute("CREATE TABLE coursedrop (course_combo text, Count integer)")
        except:
            cursor.execute("CREATE TABLE coursedrop (course_combo text, Count integer)")
        cursor.executemany("INSERT INTO coursedrop VALUES (?,?)",postList)
        cursor.execute("SELECT course_combo, SUM(Count) FROM coursedrop GROUP BY course_combo")
        postQry = cursor.fetchall()
        postEntropy = shannonEntropy(postQry)
        postCount = 0
        for item in postQry:
            postCount += item[1]
        changeVals = []
        for k in range(len(i)):
            oldSpots = []
            iterTemp = itertools.combinations(i,k+1)
            while True:
                try: oldSpots.append(iterTemp.next())
                except: break
            for l in oldSpots:
                for m in postQry:
                    mList = list(m[0])
                    for n in l:
                        mList[n] = '1'
                    oldString = ''
                    for p in mList:
                        oldString+=p
                    if m[1]>=k and oldString in preCombos:
                        changeVals.append(oldString)
                    elif (m[0] in nonUniqueList) and oldString in preCombos:
                        changeVals.append(oldString)
        #print "Length of ChangeVals: "+str(len(changeVals))
        if len(changeVals)>0:
            postEntList.append((i,preEntropy-postEntropy,changeVals))
    if len(postEntList) == 0:
        return []
    first = True
    low = (99,99,[])
    for n in postEntList:
        if n[1]<low[1] and n[1] > 0.0:
            low = n
    return low

In [13]:
def userKanon2(cursor, tableName, userVar, courseVar, k):
    """                                                                                                                                                                                          
    cursor: sqlite cursor object                                                                                                                                                                 
    tableName: string, name of table                                                                                                                                                             
    userVar: string, name of userid variable                                                                                                                                                     
    courseVar: string, name of course variable                                                                                                                                                   
    k: minimum group size                                                                                                                                                                        
    creates a unique row record that is combo of                                                                                                                                                 
    courseid and userid, and then creates another variable                                                                                                                                       
    that says which courses someone has taken                                                                                                                                                    
    then checks for unique count of courses taken                                                                                                                                                
    and unique combinations of courses                                                                                                                                                           
    """
    courseList = courseComboUpdate(cursor,tableName,userVar,courseVar)
    value, uniqueList, nonUniqueList = uniqUserCheck(cursor,tableName,userVar,k)
    uniqUserFlag(cursor, tableName, uniqueList)
    dropNum = 1
    courseDrops = {}
    while value != 0.0 and dropNum != 16:  
        print "DropNum: "+str(dropNum)
        print "non-anon value: "+str(value)
        courseTup = optimumDrop2(cursor, tableName, userVar, k, nonUniqueList,dropNum)
        #print "courseTup returned from OptimumDrop:"
        if len(courseTup) == 0 or len(courseTup[2])==0:
            dropNum +=1 
            print "no more changes can be made, trying "+str(dropNum)+" courses at a time"   
            return courseDrops
        #print courseTup[:2]  
        courseNums = courseTup[0]
        #print "courseNums:"
        #print courseNums
        changeVals = courseTup[2]
        print "length of changeVals"
        print len(changeVals)
        for i in courseNums:
            courseName = courseList[i]
            print "dropping courseName:"
            print courseName
            courseDrops = courseDropper2(cursor, tableName, courseVar, courseName, changeVals, courseDrops)
        courseList = courseComboUpdate(cursor,tableName,userVar,courseVar)
        value, uniqueList, nonUniqueList = uniqUserCheck(cursor,tableName,userVar,k)
        uniqUserFlag(cursor, tableName, uniqueList)
    return courseDrops

In [14]:
def courseDropper2(cursor, tableName, courseVar, courseName, changeVals, courseDict={}):
    """                                                                                                                                                                                          
    courseName: string, name of course to be dropped                                                                                                                                             
    changeVals: list of strings, values of course_combo to drop                                                                                                                                  
    courseDict: dictionary of courses and running tally of rows dropped                                                                                                                          
    drops course record where course equals courseName                                                                                                                                           
    AND uniqUserFlag = "True"                                                                                                                                                                    
    """
    delCount = 0
    #print "len of changeVals: "+str(len(changeVals))
    for val in changeVals:
        cursor.execute("SELECT SUM(Count) FROM "+tableName+" WHERE ("+courseVar+" = '"+courseName+"' AND uniqUserFlag = 'True' AND course_combo = '"+val+"')")
        qry = cursor.fetchall()
        #print "changeVal qry length:"+str(len(qry))
        if (qry[0][0]): delCount += qry[0][0]
    print "delCount: "+str(delCount)
    if delCount == 0:
        return courseDict
    if courseName in courseDict.keys():
        courseDict[courseName] += delCount
    else:
        courseDict[courseName] = delCount
    #confirm = raw_input("Confirm you want to delete "+str(delCount)+" records associated with "+courseName+" (y/n): ")
    #if confirm == 'n':
    #    return
    #elif confirm == 'y':
    for val in changeVals:
        cursor.execute("DELETE FROM "+tableName+" WHERE ("+courseVar+" = '"+courseName+"' AND uniqUserFlag = 'True' AND course_combo = '"+val+"')")
    #else:
    #    print "invalid choice, exiting function"
    return courseDict

In [15]:
def kAnonIter(cursor, tableName, k, outFile):
    """                                                                                                                                                                                          
    cursor: sqlite cursor object                                                                                                                                                                 
    tableName: string, name of table                                                                                                                                                             
    k: minimum group size                                                                                                                                                                        
    wrapper function, gets list of variables from user input,                                                                                                                                    
    updates kkey, checks for k-anonymity                                                                                                                                                         
    """
    coreVarList = qiPicker(cursor, tableName)
    optVarList = qiPicker(cursor, tableName)
    iterVarList = coreVarList
    addList = []
    kkeyUpdate(cursor, tableName, iterVarList)
    varIndex = 0
    a,b = isTableKanonymous(cursor, tableName,k)
    results = [('core',b)]
    for var in optVarList:
        iterVarList.append(optVarList[varIndex])
        print iterVarList
        addList.append(optVarList[varIndex])
        print addList
        results.append((addList,))
        kkeyUpdate(cursor, tableName, iterVarList)
        a,b = isTableKanonymous(cursor,tableName,k)
        varIndex += 1
        results[varIndex] += (b,)
    outFile.write(str(results))
    return results

#### Name the file containing the data, name the database, and name commonly-used database variables. NOTE: make updates here if file specification changes

In [16]:
file = "HMXPC13_DI_binned_061714_k3_QIall.csv"
table = "source"
userVar = "user_id"
courseVar = "course_id"
countryVar = "final_cc"
k=5

In [17]:
#choose a name for the database and then connect to it
db = 'kaPC_1-17-4-17-14-3.db'
c = dbOpen(db)

#### Load data into SQLite database

In [19]:
 
def sourceLoad(cursor, fname, tableName):
    """
    cursor: sqlite3 cursor object
    fname: string, file name/path for loading, .csv format
    takes a .csv file and reads it into a sqlite database defined by the 
    cursor object. 
    CAUTION: will DELETE any existing table with same name
    """
    try:
        cursor.execute("DROP TABLE "+tableName)
        cursor.execute("DROP TABLE original")
    except:
        pass
    with open(fname, "rU") as inFile:
        csvIn = csv.reader(inFile)
        row = csvIn.next()
        headers = row
        tableCreate = "CREATE TABLE "+tableName+" ("
        tableInsert = "INSERT INTO "+tableName+" VALUES ("
        for col in headers[:-1]:
            tableCreate += col+" text, "
            tableInsert += "?,"
        tableCreate += headers[-1]+" text, kkey text)"
        cursor.execute(tableCreate)

        tableInsert += "?, ?)"
        #varList = qiPicker(cursor, tableName)
        for row in csvIn:
            if (row[25] == 'instructor') or (row[25] == 'staff' ):
                continue
            #for i in varList:
            #    lastVar += row[int(i[0])]
            row[14] = splitDate(row[14])
            row[15] = splitDate(row[15])
            trow = tuple(row)
            trow += ("",)
            cursor.execute(tableInsert, trow)
            
    cursor.execute("ALTER TABLE "+tableName+" ADD COLUMN Count integer")
    cursor.execute("UPDATE "+tableName+" SET Count =1")
    #return varList


c.execute('DROP TABLE IF EXISTS '+table)
sourceLoad(c,file,table)

IOError: [Errno 2] No such file or directory: 'HMXPC13_DI_binned_061714_k3_QIall.csv'

#### Load data into another table to make comparisons to the original data

In [20]:
c.execute('DROP TABLE IF EXISTS original')
c.execute('CREATE TABLE original AS SELECT * from ' + table) 

OperationalError: no such table: source

#### Get sqlite table names

In [501]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[(u'coursedrop',),
 (u'idhash',),
 (u'userkcheck',),
 (u'kacheck',),
 (u'source',),
 (u'original',)]

#### Load column names into a variable called varList

In [502]:
c.execute("Pragma table_info("+table+")")
varList = c.fetchall()
varList

[(0, u'course_id', u'text', 0, None, 0),
 (1, u'user_id', u'text', 0, None, 0),
 (2, u'username', u'text', 0, None, 0),
 (3, u'registered', u'text', 0, None, 0),
 (4, u'viewed', u'text', 0, None, 0),
 (5, u'explored', u'text', 0, None, 0),
 (6, u'certified', u'text', 0, None, 0),
 (7, u'ip', u'text', 0, None, 0),
 (8, u'cc_by_ip', u'text', 0, None, 0),
 (9, u'cc_address', u'text', 0, None, 0),
 (10, u'LoE', u'text', 0, None, 0),
 (11, u'YoB', u'text', 0, None, 0),
 (12, u'gender', u'text', 0, None, 0),
 (13, u'grade', u'text', 0, None, 0),
 (14, u'start_time', u'text', 0, None, 0),
 (15, u'last_event', u'text', 0, None, 0),
 (16, u'nevents', u'text', 0, None, 0),
 (17, u'modal_ip', u'text', 0, None, 0),
 (18, u'modal_ip_cc', u'text', 0, None, 0),
 (19, u'final_cc', u'text', 0, None, 0),
 (20, u'final_cc_source', u'text', 0, None, 0),
 (21, u'ndays_act', u'text', 0, None, 0),
 (22, u'nplay_video', u'text', 0, None, 0),
 (23, u'nchapters', u'text', 0, None, 0),
 (24, u'nforum_posts', u't

#### Add indices

In [503]:
varIndex(c,table,courseVar)
varIndex(c,table,userVar)

In [504]:
c.execute("CREATE INDEX "+courseVar+"_idx2 ON original ("+courseVar+")")
c.execute("CREATE INDEX "+userVar+"_idx2 ON original ("+userVar+")")

#### Get initial count of records loaded

In [505]:
c.execute("SELECT SUM(Count) FROM "+table)
c.fetchall()

[(419174,)]

#### Map country codes to country names, load table of country name to continent mappings

In [506]:
countryNamer(c,table,countryVar)
contImport(c, table, "country_continent", countryVar+"_cname")

Err '' on: cc=
Err 'A1' on: cc=A1
Err 'A2' on: cc=A2
Err 'AP' on: cc=AP
Err 'EU' on: cc=EU


#### Delete staff

In [507]:
c.execute("DELETE FROM "+table+" WHERE (roles = 'instructor' or roles = 'staff')")

In [508]:
c.execute("DELETE FROM original WHERE (roles = 'instructor' or roles = 'staff')")

#### Generate anonymous userIDs, choose prefix that will describe the data release, here 'MHxPC13' refers to MITx/HarvardX Person-Course AY2013

In [509]:
idGen(c,table,userVar,"MHxPC13")

ids: 363425


#### Get initial entropy reading

In [510]:
addColumn(c,table,'entropy')
varIndex(c,table,'entropy')
kkeyUpdate(c, table,varList,'entropy')

In [511]:
qry = selUnique(c,table,'entropy')

In [512]:
beginEntropy = shannonEntropy(qry)
beginEntropy

18.677189708501597

#### Create utility Matrix (both for unmodified dataset and current dataset)

In [513]:
utilVars = varList[4:7]+[varList[13]]+[varList[16]]+varList[21:25]
utilVars

[(4, u'viewed', u'text', 0, None, 0),
 (5, u'explored', u'text', 0, None, 0),
 (6, u'certified', u'text', 0, None, 0),
 (13, u'grade', u'text', 0, None, 0),
 (16, u'nevents', u'text', 0, None, 0),
 (21, u'ndays_act', u'text', 0, None, 0),
 (22, u'nplay_video', u'text', 0, None, 0),
 (23, u'nchapters', u'text', 0, None, 0),
 (24, u'nforum_posts', u'text', 0, None, 0)]

In [514]:
preUmatrix = utilMatrix(c,"original",utilVars)

In [515]:
preUmatrix

Entropy        Mean         SD
viewed        0.9711086   0.5997295  0.4899531
explored       0.422227  0.08578061  0.2800398
certified     0.2347492  0.03836593  0.1920781
grade          1.461635   0.0447237  0.1789314
nevents        5.699504    489.2225   1780.657
ndays_act      3.183227    7.011011   13.73271
nplay_video    1.588071    302.4309   1152.513
nchapters      2.826516    4.388149   6.050422
nforum_posts  0.6015475   0.4572636   8.060734

[9 rows x 3 columns]

In [516]:
uMatrix = utilMatrix(c,table,utilVars)

In [517]:
uMatrix

Entropy        Mean         SD
viewed        0.9711086   0.5997295  0.4899531
explored       0.422227  0.08578061  0.2800398
certified     0.2347492  0.03836593  0.1920781
grade          1.461635   0.0447237  0.1789314
nevents        5.699504    489.2225   1780.657
ndays_act      3.183227    7.011011   13.73271
nplay_video    1.588071    302.4309   1152.513
nchapters      2.826516    4.388149   6.050422
nforum_posts  0.6015475   0.4572636   8.060734

[9 rows x 3 columns]

In [518]:
uMatrix - preUmatrix
#removed rows for user k-anonymity

Entropy Mean SD
viewed             0    0  0
explored           0    0  0
certified          0    0  0
grade              0    0  0
nevents            0    0  0
ndays_act          0    0  0
nplay_video        0    0  0
nchapters          0    0  0
nforum_posts       0    0  0

[9 rows x 3 columns]

#### Establish user-wise k-anonymity (the removal of registrations that uniquely identify someone based on combination of courses registered for)

In [519]:
courseDrops = userKanon2(c, table, userVar, courseVar, k)

generating course list
23:00:15.061596
creating/overwriting course_combo
23:00:40.748037
no. of unique users to update: 363425
23:01:09.359834


In [520]:
for course in courseDrops.keys():
    print "Dropped "+str(courseDrops[course])+" rows for course "+course

In [521]:
c.execute("SELECT SUM(Count) FROM "+table+" WHERE uniqUserFlag = 'True'")                                                                                                                
qry = c.fetchall()                                                                                                                           
print "Deleted "+str(qry[0][0])+" additional records for users with unique combinations of courses.\n"                                                                                   
c.execute("DELETE FROM "+table+" WHERE uniqUserFlag = 'True'")                                                                                                                                   

Deleted None additional records for users with unique combinations of courses.



In [522]:
kkeyUpdate(c, table,varList[:26],'entropy')

In [523]:
qry = selUnique(c,table,'entropy')
tmpEntropy = shannonEntropy(qry)

In [524]:
tmpEntropy

18.677189708501597

In [525]:
entChg = 2**beginEntropy - 2**tmpEntropy

In [528]:
entChg
#This one after User-K-Anonymity

0.0

#### Replace country names with continent names

In [527]:
initContVal = 5000
contSwap(c,table,"final_cc_cname","continent",initContVal)
#outFile.write("Inserting continent names for countries with fewer than "+str(initContVal)+"\n")

countries: 229
categories after swap: 22


#### Make gender variable that treats NA and missing as same

In [529]:
try:
    addColumn(c,table,"gender_DI")
    varIndex(c,table,"gender_DI")
    simpleUpdate(c,table,"gender_DI","NULL")
    c.execute("UPDATE "+table+" SET gender_DI = gender")
    c.execute("UPDATE "+table+" SET gender_DI = '' WHERE gender_DI = 'NA'")
except:
    c.execute("UPDATE "+table+" SET gender_DI = gender")
    c.execute("UPDATE "+table+" SET gender_DI = '' WHERE gender_DI = 'NA'")

#### Get k-anonymity reading

In [530]:
status, value = kAnonWrap(c,table,k)
print "Percent of records that will need to be deleted to be k-anonymous: "+str(value)+"\n"
#outFile.write( "Percent of records that will need to be deleted to be k-anonymous: "+str(value)+"\n")

Please choose the QI variables from the list below:
0. course_id
1. user_id
2. username
3. registered
4. viewed
5. explored
6. certified
7. ip
8. cc_by_ip
9. cc_address
10. LoE
11. YoB
12. gender
13. grade
14. start_time
15. last_event
16. nevents
17. modal_ip
18. modal_ip_cc
19. final_cc
20. final_cc_source
21. ndays_act
22. nplay_video
23. nchapters
24. nforum_posts
25. roles
26. kkey
27. Count
28. final_cc_cname
29. continent
30. userid_DI
31. entropy
32. course_combo
33. uniqUserFlag
34. final_cc_cname_DI
35. gender_DI
Enter your choices by number, separated by commas: 0,10,11,35,34
Percent of records that will need to be deleted to be k-anonymous: 0.122960870665



#### Check k-anonymity for records with some null values

In [531]:
print "checking k-anonymity for records with some null values"
print datetime.datetime.now().time()
#outFile.write("checking k-anonymity for records with some null values\n")
#outFile.write(str(datetime.datetime.now().time())+"\n")
iterKcheck(c,table,k)

checking k-anonymity for records with some null values
23:31:08.995990
Please choose the QI variables from the list below:
0. course_id
1. user_id
2. username
3. registered
4. viewed
5. explored
6. certified
7. ip
8. cc_by_ip
9. cc_address
10. LoE
11. YoB
12. gender
13. grade
14. start_time
15. last_event
16. nevents
17. modal_ip
18. modal_ip_cc
19. final_cc
20. final_cc_source
21. ndays_act
22. nplay_video
23. nchapters
24. nforum_posts
25. roles
26. kkey
27. Count
28. final_cc_cname
29. continent
30. userid_DI
31. entropy
32. course_combo
33. uniqUserFlag
34. final_cc_cname_DI
35. gender_DI
Enter your choices by number, separated by commas: 0,10,11,35,34
Checking 10001...
[(0, u'course_id'), (34, u'final_cc_cname_DI')]
23:53:45.015561
rows in qry: 110, time:
23:57:43.969309
Checking 10011...
[(0, u'course_id'), (35, u'gender_DI'), (34, u'final_cc_cname_DI')]
23:59:31.776197
rows in qry: 409, time:
00:02:11.194228
Checking 10111...
[(0, u'course_id'), (11, u'YoB'), (35, u'gender_DI'),

In [532]:
def eduClean(cursor, tableName, loeVar):
    try: 
        addColumn(cursor,tableName,loeVar+"_DI")
        varIndex(cursor,tableName,loeVar+"_DI")
    except:
        simpleUpdate(cursor,tableName,loeVar+"_DI","NULL")
    ed_dict = {'':'', 'NA':'NA','a':'Secondary','b':"Bachelor's",'el':'Less than Secondary',
               'hs':'Secondary','jhs':'Less than Secondary','learn':'','m':"Master's",'none':'',
               'other':'','p':'Doctorate','p_oth':'Doctorate','p_se':'Doctorate'}
    qry = selUnique(cursor,tableName,loeVar)
    for row in qry:
        if row[0] in ed_dict.keys():
            cursor.execute('UPDATE '+tableName+' SET '+loeVar+'_DI = "'+ed_dict[row[0]]+'" WHERE '+loeVar+' = "'+row[0]+'"')

In [533]:
eduClean(c,table,"LoE")

In [534]:
selUnique(c,table,"LoE")

[(u'', 16885),
 (u'NA', 20223),
 (u'a', 3555),
 (u'b', 140527),
 (u'el', 1870),
 (u'hs', 96684),
 (u'jhs', 10260),
 (u'learn', 1),
 (u'm', 93454),
 (u'none', 2914),
 (u'other', 13634),
 (u'p', 3535),
 (u'p_oth', 7320),
 (u'p_se', 8312)]

In [535]:
selUnique(c,table,"LoE_DI")

[(u'', 33434),
 (u"Bachelor's", 140527),
 (u'Doctorate', 19167),
 (u'Less than Secondary', 12130),
 (u"Master's", 93454),
 (u'NA', 20223),
 (u'Secondary', 100239)]

In [536]:
#change 0 values to text in order to exclude them from the binning procedure
c.execute("UPDATE "+table+" SET nforum_posts = '0' WHERE nforum_posts = 'zero'")

#### The Tailfinder function can help to group a long tail of one variable into a text field (see more documentation in the de_id_functions.py file)

In [537]:
tailFinder(c,table,"nforum_posts",1)

Would you like to trim the tails? (y/n): y
High (h), Low (l), or Both (b)?: h
Choose the high tail: 7
High tail for nforum_posts: 7


In [260]:
numBinner(c,table,"nforum_posts",bw=1)

Copy into (n)ew variable or (o)verwrite?: n
column nforum_posts_DI already exists, overwriting...


In [538]:
binAvg(c,table,"nforum_posts_DI","nforum_posts")

In [539]:
selUnique(c,table,"nforum_posts_DI")

[(u'0', 390647),
 (u'1', 9801),
 (u'2', 4932),
 (u'3', 2843),
 (u'4', 2129),
 (u'5', 1439),
 (u'6', 1101),
 (u'>= 7', 6282)]

In [540]:
tailFinder(c,table,"YoB",1)

non int value: NA, skipping
Would you like to trim the tails? (y/n): y
High (h), Low (l), or Both (b)?: l
Choose the low tail: 1931
Low tail for YoB: 1931


In [541]:
numBinner(c,table,"YoB_DI",bw=1)

Copy into (n)ew variable or (o)verwrite?: n


In [542]:
selUnique(c,table,"YoB_DI")

[(None, 42152),
 (u'1932', 63),
 (u'1933', 55),
 (u'1934', 58),
 (u'1935', 94),
 (u'1936', 112),
 (u'1937', 155),
 (u'1938', 150),
 (u'1939', 186),
 (u'1940', 201),
 (u'1941', 232),
 (u'1942', 309),
 (u'1943', 360),
 (u'1944', 431),
 (u'1945', 408),
 (u'1946', 530),
 (u'1947', 678),
 (u'1948', 648),
 (u'1949', 767),
 (u'1950', 907),
 (u'1951', 984),
 (u'1952', 1035),
 (u'1953', 1077),
 (u'1954', 1224),
 (u'1955', 1457),
 (u'1956', 1535),
 (u'1957', 1573),
 (u'1958', 1670),
 (u'1959', 1769),
 (u'1960', 2200),
 (u'1961', 2225),
 (u'1962', 2315),
 (u'1963', 2475),
 (u'1964', 2633),
 (u'1965', 2812),
 (u'1966', 2919),
 (u'1967', 3093),
 (u'1968', 3440),
 (u'1969', 3625),
 (u'1970', 4260),
 (u'1971', 4168),
 (u'1972', 4699),
 (u'1973', 4822),
 (u'1974', 5362),
 (u'1975', 5844),
 (u'1976', 6524),
 (u'1977', 7161),
 (u'1978', 8031),
 (u'1979', 9062),
 (u'1980', 10707),
 (u'1981', 10991),
 (u'1982', 12443),
 (u'1983', 13469),
 (u'1984', 15261),
 (u'1985', 17100),
 (u'1986', 18553),
 (u'1987', 

In [544]:
kAnonWrap(c,table,k)

Please choose the QI variables from the list below:
0. course_id
1. user_id
2. username
3. registered
4. viewed
5. explored
6. certified
7. ip
8. cc_by_ip
9. cc_address
10. LoE
11. YoB
12. gender
13. grade
14. start_time
15. last_event
16. nevents
17. modal_ip
18. modal_ip_cc
19. final_cc
20. final_cc_source
21. ndays_act
22. nplay_video
23. nchapters
24. nforum_posts
25. roles
26. kkey
27. Count
28. final_cc_cname
29. continent
30. userid_DI
31. entropy
32. course_combo
33. uniqUserFlag
34. final_cc_cname_DI
35. gender_DI
36. course_id_NF
37. LoE_NF
38. YoB_NF
39. gender_DI_NF
40. final_cc_cname_DI_NF
41. nullSum
42. kCheckFlag
43. nullkkey
44. LoE_DI
45. nforum_posts_DI
46. nforum_posts_DI_avg
47. YoB_DI
48. YoB_DI_DI
Enter your choices by number, separated by commas: 0,34,35,44,47


(False, 0.10952981301341685)

In [545]:
lDiversity(c,table,"kkey","grade")

#### Needed an incomplete flag for internally inconsistent records. This is described more in the documentation with the data release.

In [546]:
addColumn(c,table,"incomplete_flag")

In [547]:
varIndex(c,table,"incomplete_flag")

In [548]:
c.execute("SELECT SUM(Count) FROM source WHERE nevents = '' AND nchapters != ''")
qry = c.fetchall()
print qry
c.execute("SELECT SUM(Count) FROM source WHERE nevents = '' AND nforum_posts != '0'")
qry = c.fetchall()
print qry
c.execute("SELECT SUM(Count) FROM source WHERE nevents = '' AND ndays_act != ''")
qry = c.fetchall()
print qry

[(None,)]
[(None,)]
[(None,)]


In [549]:
c.execute("UPDATE source SET incomplete_flag = '1' WHERE nevents = '' AND nchapters != ''")

In [550]:
c.execute("UPDATE source SET incomplete_flag = '1' WHERE nevents = '' AND nforum_posts != '0'")

In [551]:
c.execute("UPDATE source SET incomplete_flag = '1' WHERE nevents = '' AND ndays_act != ''")

In [552]:
c.execute("SELECT * FROM source WHERE incomplete_flag = '1'")

In [553]:
qry = c.fetchall()

In [554]:
len(qry)

0

In [555]:
c.execute("Pragma table_info(source)")
varList = c.fetchall()
varList

[(0, u'course_id', u'text', 0, None, 0),
 (1, u'user_id', u'text', 0, None, 0),
 (2, u'username', u'text', 0, None, 0),
 (3, u'registered', u'text', 0, None, 0),
 (4, u'viewed', u'text', 0, None, 0),
 (5, u'explored', u'text', 0, None, 0),
 (6, u'certified', u'text', 0, None, 0),
 (7, u'ip', u'text', 0, None, 0),
 (8, u'cc_by_ip', u'text', 0, None, 0),
 (9, u'cc_address', u'text', 0, None, 0),
 (10, u'LoE', u'text', 0, None, 0),
 (11, u'YoB', u'text', 0, None, 0),
 (12, u'gender', u'text', 0, None, 0),
 (13, u'grade', u'text', 0, None, 0),
 (14, u'start_time', u'text', 0, None, 0),
 (15, u'last_event', u'text', 0, None, 0),
 (16, u'nevents', u'text', 0, None, 0),
 (17, u'modal_ip', u'text', 0, None, 0),
 (18, u'modal_ip_cc', u'text', 0, None, 0),
 (19, u'final_cc', u'text', 0, None, 0),
 (20, u'final_cc_source', u'text', 0, None, 0),
 (21, u'ndays_act', u'text', 0, None, 0),
 (22, u'nplay_video', u'text', 0, None, 0),
 (23, u'nchapters', u'text', 0, None, 0),
 (24, u'nforum_posts', u't

In [556]:
c.execute("SELECT uniqUserFlag FROM source LIMIT 1")
c.fetchall()

[(u'False',)]

In [560]:
kkeyList = []
kkeyList.append(varList[0]) # course ID
kkeyList.append(varList[34]) # country name DI
kkeyList.append(varList[35]) # gender DI
kkeyList.append(varList[44]) # LoE DI
kkeyList.append(varList[46]) # nforum posts DI
kkeyList.append(varList[47]) # YoB DI
kkeyList 

[(0, u'course_id', u'text', 0, None, 0),
 (34, u'final_cc_cname_DI', u'text', 0, None, 0),
 (35, u'gender_DI', u'text', 0, None, 0),
 (44, u'LoE_DI', u'text', 0, None, 0),
 (47, u'YoB_DI', u'text', 0, None, 0)]

In [561]:
kkeyUpdate(c,table,kkeyList)

In [562]:
c.execute("SELECT SUM(Count), kkey FROM source GROUP BY kkey")
qry2 = c.fetchall()
#lessThanK = []
#badCount = 0
c.execute("UPDATE "+table+" SET kCheckFlag = 'False'")
for row in qry2:
    if row[0] >= k:
        c.execute('UPDATE '+table+' SET kCheckFlag = "True" WHERE kkey = "'+row[1]+'"')

In [563]:
selUnique(c,table,"kCheckFlag")

[(u'False', 42855), (u'True', 376319)]

In [564]:
print k

5


#### The fateful step where non-k-anonymous records are removed.

In [565]:
c.execute("DELETE FROM source WHERE kCheckFlag = 'False'")

#### Be careful to only export the columns you are ok with others seeing. Don't export IP address, original user_id, etc.

In [566]:
csvExport(c,table,"HMXPC13_DI_binned_061714_k5_QIall.csv")

Please choose the QI variables from the list below:
0. course_id
1. user_id
2. username
3. registered
4. viewed
5. explored
6. certified
7. ip
8. cc_by_ip
9. cc_address
10. LoE
11. YoB
12. gender
13. grade
14. start_time
15. last_event
16. nevents
17. modal_ip
18. modal_ip_cc
19. final_cc
20. final_cc_source
21. ndays_act
22. nplay_video
23. nchapters
24. nforum_posts
25. roles
26. kkey
27. Count
28. final_cc_cname
29. continent
30. userid_DI
31. entropy
32. course_combo
33. uniqUserFlag
34. final_cc_cname_DI
35. gender_DI
36. course_id_NF
37. LoE_NF
38. YoB_NF
39. gender_DI_NF
40. final_cc_cname_DI_NF
41. nullSum
42. kCheckFlag
43. nullkkey
44. LoE_DI
45. nforum_posts_DI
46. nforum_posts_DI_avg
47. YoB_DI
48. YoB_DI_DI
49. incomplete_flag
Enter your choices by number, separated by commas: 0,30,3,4,5,6,34,44,48,35,13,14,15,16,21,22,23,25,49


# Stats on Original File

In [ ]:
dbClose(c)
db = 'kaPC_1-17-4-17-14-orig.db'
c = dbOpen(db)

In [ ]:
c.execute("Pragma table_info(source)")
c.fetchall()

In [ ]:
c.execute("UPDATE source SET Count = 1")

In [ ]:
c.execute("SELECT SUM(Count) FROM source")
total = c.fetchall()[0][0]
total

In [ ]:
view_qry = selUnique(c,table,"viewed")
view_dic = {}
for row in view_qry:
    view_dic[row[0]] = float(row[1])/float(total)
view_dic

In [ ]:
exp_qry = selUnique(c,table,"explored")
exp_dic = {}
for row in exp_qry:
    exp_dic[row[0]] = float(row[1])/float(total)
exp_dic

In [ ]:
cert_qry = selUnique(c,table,"certified")
cert_dic = {}
for row in cert_qry:
    cert_dic[row[0]] = float(row[1])/float(total)
cert_dic

In [ ]:
gen_qry = selUnique(c,table,"gender")
gen_dic = {}
gen_total = total
for row in gen_qry:
    if row[0] == '' or row[0] == 'NA' or row[0] == 'o':
        gen_total -= row[1]
    else:
        gen_dic[row[0]] = float(row[1])/float(gen_total)
gen_dic

In [ ]:
age_qry = selUnique(c,table,"YoB")
num = 0
denom = 0
for row in age_qry:
    try: age = 2013 - int(row[0])
    except: continue
    num += age * row[1]
    denom += row[1]
avg_age = float(num)/float(denom)
avg_age

## Stats on De-identified file

In [ ]:
dbClose(c)
db = 'kaPC_1-17-4-17-14-3.db'
c = dbOpen(db)

In [ ]:
c.execute("SELECT SUM(Count) FROM source")
total = c.fetchall()[0][0]
total

In [ ]:
view_qry = selUnique(c,table,"viewed")
view_dic = {}
for row in view_qry:
    view_dic[row[0]] = float(row[1])/float(total)
view_dic

In [ ]:
exp_qry = selUnique(c,table,"explored")
exp_dic = {}
for row in exp_qry:
    exp_dic[row[0]] = float(row[1])/float(total)
exp_dic

In [ ]:
cert_qry = selUnique(c,table,"certified")
cert_dic = {}
for row in cert_qry:
    cert_dic[row[0]] = float(row[1])/float(total)
cert_dic

In [ ]:
gen_qry = selUnique(c,table,"gender")
gen_dic = {}
gen_total = total
for row in gen_qry:
    if row[0] == '' or row[0] == 'NA' or row[0] == 'o':
        gen_total -= row[1]
    else:
        gen_dic[row[0]] = float(row[1])/float(gen_total)
gen_dic

In [ ]:
age_qry = selUnique(c,table,"YoB")
num = 0
denom = 0
for row in age_qry:
    try: age = 2013 - int(row[0])
    except: continue
    num += age * row[1]
    denom += row[1]
avg_age = float(num)/float(denom)
avg_age

In [ ]:
c.execute("Pragma database_list")
c.fetchall()

In [ ]:
selUnique(c,table,"YoB")

In [ ]:
uMatrix - preUmatrix
#This one taken after K-Anonymous

#### Good to close the db between uses, it compacts the data and prevents an error if you leave a cursor dangling.

In [107]:
dbClose(db)

AttributeError: 'str' object has no attribute 'execute'